In [1]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1', 'https://qlawsdl001038a.ad.datalake.foc.zone:8443/gateway/dsx/livy/v1']


In [2]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g", 
 "driverMemory": "8g", "proxyUser": "aliu-", "driverCores": 1, 
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [3]:
%spark add -s servicingdata -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
23011,application_1590030838276_75250,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [4]:
%%spark

print(spark.version)

2.3.0.2.6.5.0-292

In [5]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit, countDistinct, max, min, desc
from pyspark.sql import DataFrameStatFunctions as statFunc
import pyspark.sql.types as T
from functools import reduce

from os.path import expanduser, join, abspath
import time
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

spark = SparkSession.builder.getOrCreate()

In [6]:
%%spark

csv_path = '/dev/projects/retention_models/csv_data/'
#hpi_data_save_path = '/dev/projects/refi_lead_generation/datasets_old/'
monthly_path = '/dev/projects/retention_models/monthly_snapshot/'
monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'
hpi_path = '/dev/projects/retention_models/hpi/'

#Refi
model_save_path_refi = '/dev/projects/retention_models/refi_payoff/models/'
result_path_refi = '/dev/projects/retention_models/refi_payoff/training/results/'

#Pur
model_save_path_pur = '/dev/projects/retention_models/purchase_payoff/models/'
result_path_pur = '/dev/projects/retention_models/purchase_payoff/training/results/movermodel/'
result_path_pur_6 = '/dev/projects/retention_models/purchase_payoff/training/results/mover_6/'
result_path_pur_12 = '/dev/projects/retention_models/purchase_payoff/training/results/mover_12/'
result_path_pur_all = '/dev/projects/retention_models/purchase_payoff/training/results/'

#prod
prod_pur_path = '/prod/projects/model_conformed/RocketScience/LeadGen/PurLeadGen/1.0/'
prod_refi_path = '/prod/projects/model_conformed/RocketScience/LeadGen/RefiLeadGen/1.0/'
file_path = '/prod/projects/model_conformed/RocketScience/LeadGen/Currentfile/'

# GCID

In [7]:
%%spark

df = spark.read.csv(os.path.join(csv_path, 'gcidservicingloans_20200603.csv'), header=True)  #change the file name

df.write.mode('overwrite').saveAsTable('data_science_sandbox.gcidservicingloans')


# Loanlist

In [7]:
%%spark

# df = spark.read.csv(os.path.join(csv_path, 'servicing_loanlist_202005.csv'), header=True)  #change the file name

# print(df.count())
# df.show(2)

## (Before running new month) check left-portfolio data

In [7]:
%%spark

df_list = spark.read.csv(os.path.join(csv_path, 'servicing_loanlist_202005.csv'), header=True)

## right to delete loans have been removed in SQL file

print(df_list.count())
df_list.show(1)

1889579
+-------------------+----------+
|servicecalendardate|     ln_no|
+-------------------+----------+
|         2020-05-31|3368360274|
+-------------------+----------+
only showing top 1 row

In [8]:
%%spark

df_book = spark.sql('''
select distinct servicecalendardate, ln_no
from data_science_sandbox.servicing_dbo_payoff_source_purchase_client_crdtrs 
where servicecalendardate = '{0}'
and leftportfoliodate is null
'''.format('2020-05-31'))

df_book.count()

1904403

In [9]:
%%spark

df_book.show(1)

+-------------------+----------+
|servicecalendardate|     ln_no|
+-------------------+----------+
|         2020-05-31|3308906405|
+-------------------+----------+
only showing top 1 row

In [10]:
%%spark

df_book.groupBy('servicecalendardate').count().orderBy(col('servicecalendardate').desc()).show(10)

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2020-05-31|1904403|
+-------------------+-------+

In [11]:
%%spark

df1 = df_book.join(df_list, on='ln_no', how='inner')

df1.count()

1889579

## 1. Read Servicing

In [12]:
%%spark

def retrieve_dataset(servicedate, loanlist_file):
    
    spark.sql('use data_science_sandbox') 
    df_raw = spark.sql('select * from servicing_dbo_payoff_source_purchase_client_crdtrs where leftportfoliodate is null')\
                    .where(col('servicecalendardate') == servicedate)
    
    #servicing book loan list, to remove the loans already left portfolio
    df_list = spark.read.csv(os.path.join(csv_path, loanlist_file), header=True)
    
    df_ser = df_raw.join(df_list, on=['servicecalendardate', 'ln_no'], how='inner').drop_duplicates()
    
    return df_ser

In [13]:
%%spark

def retrieve_dataset_dw(servicedate):
    
    spark.sql('use data_science_sandbox') 
    df_raw = spark.sql('select * from servicing_dbo_payoff_source_purchase_client_crdtrs')\
                    .where(col('servicecalendardate') == servicedate)\
                    .drop_duplicates()
    
    return df_raw

In [14]:
%%spark

Date = '2020-05-31'
df_ser = retrieve_dataset(servicedate = Date, loanlist_file='servicing_loanlist_202005.csv')

In [15]:
%%spark

df_ser.groupby('servicecalendardate').agg(F.countDistinct('ln_no'), F.count('ln_no')).show()

+-------------------+---------------------+------------+
|servicecalendardate|count(DISTINCT ln_no)|count(ln_no)|
+-------------------+---------------------+------------+
|         2020-05-31|              1889579|     1889579|
+-------------------+---------------------+------------+

In [16]:
%%spark

## match rate with experian
df_ser.where((col('mosaichousehold').isNotNull()) & (col('mosaichousehold') != '')).count()

903978

In [25]:
%%spark

## match rate with experian
df_ser.where((col('personnum_unit').isNotNull()) & (col('personnum_unit') != '')).count()

0

In [17]:
%%spark

df_ser.show(1)

+-------------------+----------+-------------------+-------------------+--------------------+-------------------+-----------------+-----------------+-----------+--------------------+-----------+---------------+-------------------+----------+------------+-------------+-------------+-----+-----------------+----------+---------+-----------------+-----------+------------+-----------------------+-----------------------+----+--------+-----------+------------------+----------------+----------------------+-----------+---------+------------+-----------------+--------------+--------------+---------------+--------------+------------------+---------------+--------------+-----------+------------+--------+---------+--------------+-------------------+------------------+-----------+------+-------------+-------------+----------------+--------------+---------------------+----------+---------------+----------+------------------+---------+-----------------+--------------------+--------------+-------------

In [18]:
%%spark

df_ser.groupby('p_age_31_50').count().show()

+-----------+-------+
|p_age_31_50|  count|
+-----------+-------+
|          1| 294620|
|          6|    115|
|          3|  47265|
|          5|   1077|
|          4|   8843|
|          2| 180819|
|          0|1356840|
+-----------+-------+

## 2. Preprocess Data

### (1) Selected columns

In [19]:
%%spark

df_new = df_ser.select('servicecalendardate', 'og_note_dt', 'ln_no',
                        'loantypedescription', 'unpaidbalfirstsecond', 'loanamortizationtype', 'og_mtg_am',
                        'ln_purpose_type', 'og_occupy_stat_type',
                        'ln_ann_int_rt', 'ln_tr', 'ln_monthly_pmt_am', 'ln_eloc_cd',
                        'orig_ltv_ratio_fc', 'pr_value_am', 'investornameshort',
                        'issingleborrower', 'liveyears', 'pr_zip_cd',
                        'lengthresidence', 'personnum_unit', 'estimatedhomevalue', 'mosaichousehold', 'homebedroomcnt',
                        'p1_combinedage', 'p_age_31_50', 'p_age_50more', 'p_edu_hs', 'p_married', 'quarteryearkey',
                        'mostrecentcredittriggerage', 'triggercnt1monthprior', 'triggercnt2monthprior', 'triggercnt3monthprior'
                       )

#'gcid'

### (2) Market Data

Lock2close should update the market rate daily. Check the latest update date to see if there are values in the data lake.

In [20]:
%%spark

t1 = spark.sql('select * from lock2close_raw_access.dbo_marketdatafact_l2c_orc')
t1.agg(F.max('dateid')).show()

+-----------+
|max(dateid)|
+-----------+
| 2020-06-09|
+-----------+

In [21]:
%%spark

def retrieve_marketdatafact():
    """Retrieve market interest rates from DL"""
    
    df = spark.sql('select * from lock2close_raw_access.dbo_marketdatafact_l2c_orc')\
                .where(F.to_date(col('dateid'), 'yyyy-MM-dd') >= '2015-12-01')\
                .withColumn('yearmonth', col('dateid').substr(1, 7))
    
    df_market = df.groupby('yearmonth')\
                    .agg(F.min('currentfannie30yryield').alias('fannie30_min'),
                         F.min('currentfannie15yryield').alias('fannie15_min'),
                         F.min('currentginny30yryield').alias('ginny30_min'))
    
    return F.broadcast(df_market)

In [22]:
%%spark

df_market_agg = retrieve_marketdatafact()

In [23]:
%%spark

df_market_agg.filter(col('yearmonth') == '2020-06').show(5)

+---------+------------+------------+-----------+
|yearmonth|fannie30_min|fannie15_min|ginny30_min|
+---------+------------+------------+-----------+
|  2020-06|       1.618|       1.042|       1.45|
+---------+------------+------------+-----------+

In [24]:
%%spark

## check if every month has value
df_market_agg.where(F.col('yearmonth').isin('2020-05', '2020-03', '2020-04')).show(3)

+---------+------------+------------+-----------+
|yearmonth|fannie30_min|fannie15_min|ginny30_min|
+---------+------------+------------+-----------+
|  2020-05|       1.539|        1.08|       1.34|
|  2020-03|       1.468|       0.913|      1.384|
|  2020-04|       1.522|       1.094|      1.137|
+---------+------------+------------+-----------+

### (*) Update HPI (Once necessary)

In [25]:
%%spark

hpi_raw = spark.read.csv(csv_path + 'hpi_2020q1.csv', header=True)
hpi_raw.show(1)

+---+----+-------+------+--------------------+----+----+
|zip|year|quarter| index|                type| _c5| _c6|
+---+----+-------+------+--------------------+----+----+
|010|1995|      1|100.00|Native 3-Digit ZI...|null|null|
+---+----+-------+------+--------------------+----+----+
only showing top 1 row

In [26]:
%%spark

expr_mon1 = when(col('quarter')=='1', '1')\
                .otherwise(
                    when(col('quarter')=='2', '4')\
                        .otherwise(
                            when(col('quarter')=='3', '7')\
                                .otherwise('10')))
    
expr_mon2 = when(col('quarter')=='1', '2')\
                .otherwise(
                    when(col('quarter')=='2', '5')\
                        .otherwise(
                            when(col('quarter')=='3', '8')\
                                .otherwise('11')))

expr_mon3 = when(col('quarter')=='1', '3')\
                .otherwise(
                    when(col('quarter')=='2', '6')\
                        .otherwise(
                            when(col('quarter')=='3', '9')\
                                .otherwise('12')))

raw1 = hpi_raw.withColumn('month', expr_mon1)
raw2 = hpi_raw.withColumn('month', expr_mon2)
raw3 = hpi_raw.withColumn('month', expr_mon3)

hpi_new = raw1.union(raw2).union(raw3)

hpi_new.show(1)

+---+----+-------+------+--------------------+----+----+-----+
|zip|year|quarter| index|                type| _c5| _c6|month|
+---+----+-------+------+--------------------+----+----+-----+
|010|1995|      1|100.00|Native 3-Digit ZI...|null|null|    1|
+---+----+-------+------+--------------------+----+----+-----+
only showing top 1 row

In [27]:
%%spark

hpi_new.write.parquet(hpi_path + 'hpi.parquet', mode='overwrite')


In [28]:
%%spark

hpi = spark.read.parquet(hpi_path + 'hpi.parquet')

hpi.sort(desc('year')).sort(desc('quarter')).show(2)

+---+----+-------+------+--------------------+----+----+-----+
|zip|year|quarter| index|                type| _c5| _c6|month|
+---+----+-------+------+--------------------+----+----+-----+
|013|2011|      4|184.15|      MSA/MSAD Index|null|null|   10|
|011|2011|      4|183.14|Native 3-Digit ZI...|null|null|   10|
+---+----+-------+------+--------------------+----+----+-----+
only showing top 2 rows

In [30]:
%%spark

hpi.where( (col('year') == 2020) & (col('quarter') == 1) ).show(2)

+---+----+-------+------+--------------------+----+----+-----+
|zip|year|quarter| index|                type| _c5| _c6|month|
+---+----+-------+------+--------------------+----+----+-----+
|010|2020|      1|221.25|Native 3-Digit ZI...|null|null|    2|
|011|2020|      1|234.46|Native 3-Digit ZI...|null|null|    2|
+---+----+-------+------+--------------------+----+----+-----+
only showing top 2 rows

### (3) HPI


In [31]:
%%spark

  ## Loads the HPI based on 3-digit zip (for estimating home value)## Loads 

def preprocess_hpi(pullmonth, usemax=True):
    
    # If it's for current month, set usemax=True, which means the most recent as the current value.
    # If it's for historical data, set usemax=False, which means the pull month is the current value.
       
    expr_padded_month = F.lpad(col('month'), 2, '0')
    
    df_hpi = spark.read.parquet(hpi_path + 'hpi.parquet')\
                    .withColumn('hpi_yearmonth', F.concat(col('year'), F.lit('-'), expr_padded_month))\
                    .withColumn('hpi_zip', F.lpad(col('zip'), 3, '0'))
    
    df_hpi_max_date = df_hpi.groupby('hpi_zip').agg(F.max('hpi_yearmonth').alias('yearmonth_max'))\
                            .withColumn('yearmonth_pull', F.lit(pullmonth))
    
    if not usemax:
        df_hpi_curr_date = df_hpi_max_date.withColumnRenamed('yearmonth_pull', 'hpi_yearmonth')
    else:
        df_hpi_curr_date = df_hpi_max_date.withColumnRenamed('yearmonth_max', 'hpi_yearmonth')
        
    df_hpi_current = df_hpi.join(df_hpi_curr_date, on=['hpi_yearmonth', 'hpi_zip'], how='inner')\
                        .withColumnRenamed('index', 'hpi_current')\
                        .select('hpi_zip', 'hpi_current')
    
    df_hpi = df_hpi.join(df_hpi_current, on='hpi_zip', how='left')\
                .select('hpi_zip', 'hpi_yearmonth', 'index', 'hpi_current')\
                .withColumnRenamed('index', 'hpi_og')
    
    df_hpi_avg = df_hpi.groupby('hpi_yearmonth')\
                        .agg(F.avg('hpi_og').cast('float').alias('hpi_avg'), 
                             F.avg('hpi_current').cast('float').alias('hpi_current_avg'))\
                        .withColumnRenamed('hpi_yearmonth', 'hpi_yearmonth_avg')

    return F.broadcast(df_hpi), F.broadcast(df_hpi_avg)

In [32]:
%%spark

df_hpi, df_hpi_avg = preprocess_hpi(pullmonth='2020-06', usemax=True)

#historical
#df_hpi, df_hpi_avg = preprocess_hpi(pullmonth='2016-06', usemax=False)

In [33]:
%%spark

df_hpi.sort(desc('hpi_yearmonth')).show(2)

+-------+-------------+------+-----------+
|hpi_zip|hpi_yearmonth|hpi_og|hpi_current|
+-------+-------------+------+-----------+
|    124|      2020-03|285.08|     285.08|
|    447|      2020-03|168.86|     168.86|
+-------+-------------+------+-----------+
only showing top 2 rows

In [34]:
%%spark

df_hpi_avg.show(3)

+-----------------+---------+---------------+
|hpi_yearmonth_avg|  hpi_avg|hpi_current_avg|
+-----------------+---------+---------------+
|          1999-10|121.27708|      240.72694|
|          2013-05|178.59459|      240.72694|
|          2009-07|183.20718|      240.72694|
+-----------------+---------+---------------+
only showing top 3 rows

### (4) Update FICO table

In [ ]:
#%%spark

#df_fico = spark.read.csv(csv_path + 'servicing_fico_20190903.csv', header=True)\
#                .select('ln_no', col('orig_fico').cast('int'))
    
#df_fico.write.mode('overwrite').saveAsTable('data_science_sandbox.servicing_fico_aug2019_backfiled')

In [36]:
%%spark

df_fico = spark.read.csv(csv_path + 'servicing_fico_20200603.csv', header=True)\
                .select('ln_no', col('orig_fico').cast('int'))
df_fico.count()

3907106

In [37]:
%%spark

df_fico.write.mode('overwrite').saveAsTable('data_science_sandbox.servicing_fico')

In [38]:
%%spark

fico_check = spark.sql('select * from data_science_sandbox.servicing_fico')

print(fico_check.count())
fico_check.show(1)
## count should be increasing

3907106
+----------+---------+
|     ln_no|orig_fico|
+----------+---------+
|3389560443|      650|
+----------+---------+
only showing top 1 row

In [39]:
%%spark

spark.read.csv(csv_path + 'servicing_fico_20200603.csv', header=True).groupBy('insertdt').count().orderBy(desc('insertdt')).show()

+----------+-------+
|  insertdt|  count|
+----------+-------+
|2020-06-03|  83299|
|2020-05-04|  81001|
|2020-04-03|  68198|
|2020-03-03|  56059|
|2020-02-03|  57400|
|2020-01-03|  61346|
|2019-12-02|  61868|
|2019-11-05|  63390|
|2019-10-04|  52721|
|2019-09-03|  48327|
|2019-08-05|  28421|
|2019-07-12| 537306|
|2018-12-03|  29413|
|2018-11-01|  33836|
|2018-10-01|  30261|
|2018-09-04|  36346|
|2018-08-01|  32714|
|2018-07-06|  32328|
|2018-06-26|2512872|
+----------+-------+

### (5) Combine all tables

In [40]:
%%spark

def combine_tables(df, mdf = df_market_agg, df_hpi = df_hpi, df_hpi_avg=df_hpi_avg):
    
    df = df.withColumn('yearmonth', col('servicecalendardate').substr(1, 7))\
            .withColumn('og_yearmonth', col('og_note_dt').substr(1, 7))\
            .withColumn('zip3', col('pr_zip_cd').substr(1, 3))
    
    #Join market data
    df = df.join(mdf, on='yearmonth', how = 'left')
    
    #Add FICO to the table
    df_fico = spark.sql('select * from data_science_sandbox.servicing_fico')
    df = df.join(df_fico, on='ln_no', how='left')
    
    #Join the hpi data
    df = df.join(df_hpi, (df.og_yearmonth == df_hpi.hpi_yearmonth) & (df.zip3 == df_hpi.hpi_zip), how='left')\
            .withColumnRenamed('index', 'hpi_og')
            
    #Use the average to manipulate NULL hpi           
    df = df.join(df_hpi_avg, df.og_yearmonth == df_hpi_avg.hpi_yearmonth_avg, how='left')
    
    #HPI columns and HPI change
    expr_hpi_og = F.when(F.col('hpi_og').isNull(), F.col('hpi_avg')).otherwise(F.col('hpi_og'))

    expr_hpi_curr = F.when(F.col('hpi_current').isNull(), F.col('hpi_current_avg')).otherwise(F.col('hpi_current'))

    expr_hpi_change = F.when(F.col('hpi_avg').isNull(), 1.0).otherwise(expr_hpi_curr/expr_hpi_og)

    df = df.withColumn('hpi_og_exp', expr_hpi_og)\
            .withColumn('hpi_curr_exp', expr_hpi_curr)\
            .withColumn('hpi_change_exp', expr_hpi_change)
    
    return df

In [41]:
%%spark

df_all = combine_tables(df_new)

#df_all.show(1)

In [42]:
%%spark

#Save a copy for quick analysis
df_all.write.parquet(os.path.join(monthly_path, 'servicing_df_all_may20_20200610.parquet'), mode='overwrite')

### (6). Preprocess Data Function

In [43]:
%%spark

df_all = spark.read.parquet(os.path.join(monthly_path, 'servicing_df_all_may20_20200610.parquet'))

In [84]:
%%spark

def preprocess_dataset(df):
    
    ##credit trigger
    expr_ct_age = F.when(F.col('mostrecentcredittriggerage').isNull(), 0)\
                            .otherwise(
                                F.when(F.col('mostrecentcredittriggerage').isin(['NULL', 'null', '0']), 0)\
                                    .otherwise(F.col('mostrecentcredittriggerage') + 1
                                            )
                                    )
    expr_ct_1_mo = F.when(F.col('triggercnt1monthprior').isNull(), '0')\
                            .otherwise(
                                F.when(F.col('triggercnt1monthprior').isin(['NULL', 'null', '0']), '0')\
                                    .otherwise(
                                        F.when(F.col('triggercnt1monthprior') > 1, '1')\
                                            .otherwise(F.col('triggercnt1monthprior')).cast('string')
                                            )
                                    )
                                  
    expr_ct_2_mo = F.when(F.col('triggercnt2monthprior').isNull(), '0')\
                            .otherwise(
                                F.when(F.col('triggercnt2monthprior').isin(['NULL', 'null', '0']), '0')\
                                    .otherwise(
                                        F.when(F.col('triggercnt2monthprior') > 2, '2')\
                                            .otherwise(F.col('triggercnt2monthprior')).cast('string')
                                            )
                                    )
        
    expr_ct_3_mo = F.when(F.col('triggercnt3monthprior').isNull(), '0')\
                            .otherwise(
                                F.when(F.col('triggercnt3monthprior').isin(['NULL', 'null', '0']), '0')\
                                    .otherwise(
                                        F.when(F.col('triggercnt3monthprior') > 3, '3')\
                                            .otherwise(F.col('triggercnt3monthprior')).cast('string')
                                            )
                                    )
    
    ##Experian
    expr_home_value = F.when(F.col('EstimatedHomeValue').isNull(), 271225.)\
                            .otherwise(
                                F.when(F.col('EstimatedHomeValue').isin(['NULL', 'null', '0']), 271225.)\
                                    .otherwise(F.col('EstimatedHomeValue')
                                            )
                                    )
        
    expr_age_31_50 = F.when(F.col('p_age_31_50').isNull(), 0)\
                        .otherwise(
                            F.when(F.col('p_age_31_50').isin(['null', 'NULL']), 0)\
                                .otherwise(F.col('p_age_31_50').cast('int')))
    
    expr_edu_hs = F.when(F.col('p_edu_hs').isNull(), 0)\
                        .otherwise(
                            F.when(F.col('p_edu_hs').isin(['null', 'NULL']), 0)\
                                .otherwise(F.col('p_edu_hs').cast('int')))
    
    expr_married = F.when(F.col('p_married').isNull(), 0)\
                        .otherwise(
                            F.when(F.col('p_married').isin(['null', 'NULL']), 0)\
                                .otherwise(F.col('p_married').cast('int')))
        
    expr_bedroomcnt = F.when(F.col('homebedroomcnt').isNull(), 3)\
                            .otherwise(
                                F.when(F.col('homebedroomcnt').isin(['NULL', 'null', '0']), 3)\
                                    .otherwise(
                                        F.when(F.col('homebedroomcnt').isin(['1', '2', '3', '4', '5']), F.col('homebedroomcnt').cast('int'))\
                                            .otherwise(6)
                                            )
                                    )
                             
    expr_person_per_room = F.when(F.col('personnum_unit').isNull(), 3 / expr_bedroomcnt)\
                                .otherwise(
                                    F.when(F.col('personnum_unit').isin(['null', 'NULL', '0']), 3 / expr_bedroomcnt)\
                                        .otherwise(F.col('personnum_unit') / expr_bedroomcnt)
                                )
    
    #Different mosaic groups from the one in mover model!!
    expr_mosaicgroup = F.when(F.col('mosaichousehold').isin(['Null', 'null']), 'other')\
                            .otherwise(F.col('mosaichousehold').substr(0, 1))
        
    expr_mosaicgroup_refi_clust = F.when(expr_mosaicgroup.isin(['B', 'H', 'Q']), 'BHQ')\
                                   .otherwise(
                                       F.when(expr_mosaicgroup.isin(['A', 'C', 'G']), 'ACG')\
                                           .otherwise(
                                               F.when(expr_mosaicgroup.isin(['D', 'E', 'F', 'J', 'K']), 'DEFJK')\
                                                   .otherwise(
                                                       F.when(expr_mosaicgroup.isin(['I', 'L']), 'IL')\
                                                           .otherwise(
                                                               F.when(expr_mosaicgroup.isin(['M', 'N', 'O', 'P', 'R', 'S']), 'MNOPRS')\
                                                                   .otherwise('other')
                                                           )
                                                   )
                                               )
                                   )
        
    expr_mosaicgroup_pur_clust = F.when(expr_mosaicgroup.isin(['A', 'B']), 'AB')\
                                   .otherwise(
                                       F.when(expr_mosaicgroup.isin(['C', 'D', 'E']), 'CDE')\
                                           .otherwise(
                                               F.when(expr_mosaicgroup.isin(['F', 'G']), 'FG')\
                                                   .otherwise(
                                                       F.when(expr_mosaicgroup.isin(['I', 'J']), 'IJ')\
                                                           .otherwise('other')
                                                   )
                                            )
                                   )    
        
    ##Market Rate
    expr_rate_spread_min = F.when(F.col('ln_ann_int_rt').isNull(), 0.0)\
                                .otherwise(
                                    F.when(F.col('ln_ann_int_rt').isin(['null', 'NULL']), 0.0)\
                                        .otherwise(F.when(F.col('loantypedescription') == 'CONV', 
                                                         F.when(F.col('ln_tr') == '360', 
                                                                F.col('ln_ann_int_rt')*100 - F.col('fannie30_min'))\
                                                                .otherwise(F.col('ln_ann_int_rt')*100 - F.col('fannie15_min'))
                                                         )\
                                                   .otherwise(
                                                       F.when(F.col('loantypedescription').isin(['FHA', 'VA']), 
                                                              F.col('ln_ann_int_rt')*100 - F.col('ginny30_min'))\
                                                               .otherwise(F.col('ln_ann_int_rt')*100 - F.col('fannie30_min')))
                                                   )
                                            )

    # This is only for Mover V3 and before 
    expr_rate_spread_min_pur = F.when(F.col('ln_ann_int_rt').isNull(), 0.0)\
                                .otherwise(
                                    F.when(F.col('ln_ann_int_rt').isin(['null', 'NULL']), 0.0)\
                                        .otherwise(F.when(F.col('loantypedescription') == 'CONV', 
                                                         F.when(F.col('ln_tr') == '360', 
                                                                 F.col('fannie30_min') - F.col('ln_ann_int_rt')*100)\
                                                                .otherwise(F.col('fannie15_min') - F.col('ln_ann_int_rt')*100)
                                                         )\
                                                   .otherwise(
                                                       F.when(F.col('loantypedescription').isin(['FHA', 'VA']), 
                                                               F.col('ginny30_min') - F.col('ln_ann_int_rt')*100)\
                                                               .otherwise(F.col('fannie15_min')) - F.col('ln_ann_int_rt')*100)
                                                   )
                                            )

    expr_borrowerage_group = F.when(F.col('p1_combinedage').isNull(), 'median')\
                                .otherwise(
                                    F.when(F.col('p1_combinedage').isin(['Null', 'null']), 'median')\
                                        .otherwise(
                                            F.when(F.col('p1_combinedage').substr(2, 2).cast('double') > 40.0, 'old')\
                                                .otherwise(
                                                    F.when(F.col('p1_combinedage').substr(2, 2).cast('double') < 25.0, 'young')\
                                                        .otherwise('median')
                                                )
                                       )
                                )
    
    ##Servicing Data       
    expr_loan_type = F.when(F.col('loantypedescription').isin(['FHA', 'VA']), F.lower(F.col('loantypedescription')))\
                        .otherwise('conv')
    
    expr_ageinmon = F.when(F.col('og_note_dt').isNull(), 83)\
                        .otherwise(
                            F.when(F.col('og_note_dt').isin(['null', 'NULL']), 83)\
                                .otherwise(F.months_between(F.col('servicecalendardate'), F.col('og_note_dt')))
                        )        
    
    #Currentcltv (use HPI change)
    expr_og_mtg_am = when(col('og_mtg_am').isNull(), 173200.)\
                        .otherwise(
                            when(col('og_mtg_am').isin(['NULL', 'null']), 173200.)
                                .otherwise(
                                    col('og_mtg_am').cast('double')
                                    )
                            )

    expr_og_homevalue = when((col('pr_value_am').isNotNull()) & (~col('pr_value_am').isin(['null', 'NULL', '0', 0])), col('pr_value_am'))\
                            .otherwise(
                                when(col('orig_ltv_ratio_fc').isNull(), 0.)\
                                    .otherwise(
                                        when(col('orig_ltv_ratio_fc').isin(['null', 'NULL', '0', 0]), 0.)\
                                            .otherwise(expr_og_mtg_am / col('orig_ltv_ratio_fc'))
                                    )
                            )

    expr_currentcltv_cal = when(col('unpaidbalfirstsecond').isNull(), 0.)\
                                .otherwise(
                                    when(col('unpaidbalfirstsecond').isin(['null', 'NULL', '0']), 0.)\
                                        .otherwise(100 * col('unpaidbalfirstsecond') / (expr_og_homevalue * col('hpi_change_exp')))
                                )

    expr_currentcltv = when(expr_currentcltv_cal.isNull(), 70.8)\
                    .otherwise(
                        when(expr_currentcltv_cal > 150, 150)\
                            .otherwise(
                                when(expr_currentcltv_cal < 10, 10)\
                                    .otherwise(expr_currentcltv_cal)
                            )
                    )
    
    expr_orig_fico = F.when(F.col('orig_fico').isNull(), 735)\
                            .otherwise(
                                F.when(F.col('orig_fico').isin(['null', 'NULL']), 735)\
                                    .otherwise(
                                        F.when(F.col('orig_fico') > 850, 850)\
                                            .otherwise(
                                                F.when(F.col('orig_fico') < 500, 500)\
                                                    .otherwise(F.col('orig_fico').cast('int')
                                        )
                                    )
                                )
                            )
    
    #Liveyears
    expr_liveyears = F.when(F.col('lengthresidence').isNotNull(), F.col('lengthresidence'))\
                        .otherwise(
                            F.when(F.col('liveyears').isNotNull(), F.col('liveyears'))\
                                .otherwise(expr_ageinmon/12)
                            )
    
    expr_liveyears_short = F.when(expr_liveyears <= 4., expr_liveyears).otherwise(0.)
    
    expr_liveyears_long = F.when(expr_liveyears > 4, expr_liveyears).otherwise(0.)
                                  
    expr_liveyears_grp = F.when(expr_liveyears <= 4, 1.).otherwise(0.)                             
        
    expr_ln_purpose_type = F.when(F.col('ln_purpose_type').isin(['NULL', 'null', 'other']), 'refinance')\
                                .otherwise(F.lower(F.col('ln_purpose_type')))
    
    expr_og_occupy_stat_type = F.when(F.col('og_occupy_stat_type').isin(['NULL', 'null', 'other']), 'investment property')\
                            .otherwise(F.lower(F.col('og_occupy_stat_type')))

    expr_issingleborrower = F.when(F.col('issingleborrower').isNull(), 0)\
                                .otherwise(
                                    F.when(F.col('issingleborrower').isin(['NULL', 'null']), 0)
                                        .otherwise(F.col('issingleborrower')))
    
    expr_ln_tr = F.when(F.col('ln_tr').isNull(), 360)\
                    .otherwise(
                        F.when(F.col('ln_tr').isin(['NULL', 'null']), 360)\
                            .otherwise(F.col('ln_tr')))

        
    df = df.withColumn('ct_age_exp', expr_ct_age)\
        .withColumn('ct_1_exp', expr_ct_1_mo)\
        .withColumn('ct_2_exp', expr_ct_2_mo)\
        .withColumn('ct_3_exp', expr_ct_3_mo)\
        .withColumn('home_value_exp', expr_home_value)\
        .withColumn('p_edu_hs_exp', expr_edu_hs)\
        .withColumn('personnum_per_room_exp', expr_person_per_room)\
        .withColumn('mosaic_group_refi_exp', expr_mosaicgroup_refi_clust)\
        .withColumn('mosaic_group_pur_exp', expr_mosaicgroup_pur_clust)\
        .withColumn('ratespread_min_exp', expr_rate_spread_min)\
        .withColumn('ratespread_min_pur_exp', expr_rate_spread_min_pur)\
        .withColumn('loantypedescription_exp', expr_loan_type)\
        .withColumn('ageinmon_exp', expr_ageinmon)\
        .withColumn('og_mtg_am_exp', expr_og_mtg_am)\
        .withColumn('currentcltv_exp', expr_currentcltv)\
        .withColumn('orig_fico_exp', expr_orig_fico)\
        .withColumn('LiveYears_short_exp', expr_liveyears_short)\
        .withColumn('LiveYears_long_exp', expr_liveyears_long)\
        .withColumn('LiveYears_grp_exp', expr_liveyears_grp)\
        .withColumn('ln_purpose_type_exp', expr_ln_purpose_type)\
        .withColumn('og_occupy_stat_type_exp', expr_og_occupy_stat_type)\
        .withColumn('issingleborrower_exp', expr_issingleborrower)\
        .withColumn('ln_tr_exp', expr_ln_tr)\
        .withColumn('loanamortizationtype', F.lower(F.col('loanamortizationtype')))\
        .withColumn('ln_ann_int_rt', F.col('ln_ann_int_rt').cast('double'))\
        .withColumn('p_31_50_exp', expr_age_31_50)\
        .withColumn('p_married_exp', expr_married)\
        .withColumn('borrowerage_bucket', expr_borrowerage_group)

    return df

In [49]:
%%spark

def save_preprocess(df, filename):
    
    df_prep = preprocess_dataset(df)
    
    df1 = df_prep.select('servicecalendardate', 'ln_no', 'og_note_dt',
                           'ct_age_exp', 'ct_1_exp', 'ct_2_exp', 'ct_3_exp',
                           'home_value_exp', 'p_edu_hs_exp',
                           'personnum_per_room_exp', 'mosaic_group_refi_exp', 'mosaic_group_pur_exp',
                           'ratespread_min_exp', 'ratespread_min_pur_exp', 'ln_ann_int_rt',
                           'loantypedescription_exp', 'loanamortizationtype', 'ageinmon_exp',
                           'og_mtg_am_exp', 'currentcltv_exp', 'orig_fico_exp',
                           'LiveYears_short_exp', 'LiveYears_long_exp', 'LiveYears_grp_exp',
                           'ln_purpose_type_exp', 'ln_purpose_type', 'og_occupy_stat_type_exp', 'issingleborrower_exp',
                           'ln_tr_exp', 'investornameshort').drop_duplicates()
    
    df1.write.parquet(monthly_prep_path + filename, mode='overwrite')
    
# select('gcid')

In [85]:
%%spark

#def save_preprocess_old(df, filename):
    
#    df_prep = preprocess_dataset(df)
    
#    df1 = df_prep.select('servicecalendardate', 'ln_no', 'og_note_dt',
#                           'ct_age_exp', 'ct_1_exp', 'ct_2_exp', 'ct_3_exp',
#                           'home_value_exp', 'p_edu_hs_exp', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket',
#                           'personnum_per_room_exp', 'mosaic_group_refi_exp', 'mosaic_group_pur_exp',
#                           'ratespread_min_exp', 'ratespread_min_pur_exp', 'ln_ann_int_rt',
#                           'loantypedescription_exp', 'loanamortizationtype', 'ageinmon_exp',
#                           'og_mtg_am_exp', 'currentcltv_exp', 'orig_fico_exp',
#                           'LiveYears_short_exp', 'LiveYears_long_exp', 'LiveYears_grp_exp',
#                           'ln_purpose_type_exp', 'ln_purpose_type', 'og_occupy_stat_type_exp', 'issingleborrower_exp',
#                           'ln_tr_exp', 'investornameshort').drop_duplicates()
    
#    df1.write.parquet(monthly_prep_path + filename, mode='overwrite')
    
# select('gcid')

In [50]:
%%spark

save_preprocess(df_all, 'monthly_preprocessed_202005.parquet')

In [86]:
%%spark

#save_preprocess_old(df_all, 'monthly_preprocessed_202005_old.parquet')

In [51]:
%%spark

preprocessed_month_data = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202005.parquet')

print(preprocessed_month_data.count())
preprocessed_month_data.show(1)

1889579
+-------------------+----------+-------------------+----------+--------+--------+--------+--------------+------------+----------------------+---------------------+--------------------+------------------+----------------------+-------------+-----------------------+--------------------+------------+-------------+------------------+-------------+-------------------+------------------+-----------------+-------------------+---------------+-----------------------+--------------------+---------+-----------------+
|servicecalendardate|     ln_no|         og_note_dt|ct_age_exp|ct_1_exp|ct_2_exp|ct_3_exp|home_value_exp|p_edu_hs_exp|personnum_per_room_exp|mosaic_group_refi_exp|mosaic_group_pur_exp|ratespread_min_exp|ratespread_min_pur_exp|ln_ann_int_rt|loantypedescription_exp|loanamortizationtype|ageinmon_exp|og_mtg_am_exp|   currentcltv_exp|orig_fico_exp|LiveYears_short_exp|LiveYears_long_exp|LiveYears_grp_exp|ln_purpose_type_exp|ln_purpose_type|og_occupy_stat_type_exp|issingleborrower_e

In [87]:
%%spark

#preprocessed_month_data_old = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202005_old.parquet')


## 4. Mover Refi Model Scoring

### (1) Read Models

In [52]:
%%spark

refi_model_pipeline_lr = PipelineModel.load(path = model_save_path_refi+'refiV1_4_LR_20200513')
refi_model_pipeline_rf = PipelineModel.load(path = model_save_path_refi+'refiV1_4_RF_20200513')
refi_model_pipeline_gbt = PipelineModel.load(path = model_save_path_refi+'refiV1_4_GBT_20200513')

In [77]:
%%spark

## compare with the previous model

#refi_model_pipeline_lr_old = PipelineModel.load(path = model_save_path_refi+'refiV1_3_LR_20190716')
#refi_model_pipeline_rf_old = PipelineModel.load(path = model_save_path_refi+'refiV1_3_RF_20190716')
#refi_model_pipeline_gbt_old = PipelineModel.load(path = model_save_path_refi+'refiV1_3_GBT_20190716')

### (2) Rename columns to score

In [53]:
%%spark

preprocessed_month_data_refi = preprocessed_month_data             

#.withColumnRenamed('ratespread_min_exp', 'ratespread_min_refi_exp')
#.withColumnRenamed('mosaic_group_refi_exp', 'mosaic_group_exp')

### (3) Prediction

In [54]:
%%spark

def fullMonthPrediction_refi(df):
    
    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
    predictionsLR = refi_model_pipeline_lr.transform(df)
    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
                                .select('ln_no', 'og_note_dt', 'investornameshort', 'loanamortizationtype', 
                                        col('loantypedescription_exp').alias('loantypedescription'),
                                        col('ln_tr_exp').alias('loan_term'), 
                                        col('ageinmon_exp').alias('loan_age'), 
                                        col('currentcltv_exp').alias('currentcltv'),
                                        col('ln_ann_int_rt').alias('loan_interest_rate'), 
                                        col('pred').alias('logRegProb'))
    
    predictionsRF = refi_model_pipeline_rf.transform(df)
    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
                               .select('ln_no', col('pred').alias('randForProb'))

    predictionsGBT = refi_model_pipeline_gbt.transform(df)
    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
                                .select('ln_no', col('pred').alias('gbtProb'))

    final_join = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
                           .join(pred_df_gbt, on='ln_no', how='left')
     
    final_join = final_join.dropDuplicates()
    
    return final_join

In [89]:
%%spark

#def fullMonthPrediction_refi_old(df):
    
#    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
#    predictionsLR = refi_model_pipeline_lr_old.transform(df)
#    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
#                                .select('ln_no', 'og_note_dt', 'investornameshort', 'loanamortizationtype', 
#                                        col('loantypedescription_exp').alias('loantypedescription'),
#                                        col('ln_tr_exp').alias('loan_term'), 
#                                        col('ageinmon_exp').alias('loan_age'), 
#                                        col('currentcltv_exp').alias('currentcltv'),
#                                        col('ln_ann_int_rt').alias('loan_interest_rate'), 
#                                        col('pred').alias('logRegProb'))
    
#    predictionsRF = refi_model_pipeline_rf_old.transform(df)
#    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
#                               .select('ln_no', col('pred').alias('randForProb'))

#    predictionsGBT = refi_model_pipeline_gbt_old.transform(df)
#    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
#                                .select('ln_no', col('pred').alias('gbtProb'))

#    final_join = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
#                           .join(pred_df_gbt, on='ln_no', how='left')
     
#    final_join = final_join.dropDuplicates()
    
#    return final_join

In [55]:
%%spark

def dedup_result_pred_refi(pred_df):
    
    pred_df_dedup = pred_df.groupby(['ln_no', 'og_note_dt', 'loantypedescription', 'loanamortizationtype', 'investornameshort']).mean()
    
    oldcols = pred_df_dedup.schema.names
    newcols = ['ln_no', 'og_note_dt', 'loantypedescription', 'loanamortizationtype', 'investornameshort',\
               'loan_term', 'loan_age', 'currentcltv', 'loan_interest_rate',\
               'logRegProb', 'randForProb', 'gbtProb']
    pred_df_dedup = reduce(lambda pred_df_dedup, idx: pred_df_dedup.withColumnRenamed(oldcols[idx], newcols[idx]), range(len(oldcols)), pred_df_dedup)
    
    return pred_df_dedup

In [56]:
%%spark

def save_csv_refi(df_prep, filename, result_path = result_path_refi):
    
    pred = fullMonthPrediction_refi(df_prep)
    dedup = dedup_result_pred_refi(pred)
    
    dedup.coalesce(1).write.csv(result_path + filename, header=True)

In [78]:
%%spark

#def save_csv_refi_old(df_prep, filename, result_path = result_path_refi):
#    
#    pred = fullMonthPrediction_refi_old(df_prep)
#    dedup = dedup_result_pred_refi(pred)
#    
#    dedup.coalesce(1).write.csv(result_path + filename, header=True)

#### 2018

In [ ]:
%%spark

#2018.1
pred201801_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201801_dedup_refi = dedup_result_pred_refi(pred201801_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201801_dedup_refi, 'pred_refiV1_jan18_20190409.csv')

In [ ]:
%%spark

#2018.2
pred201802_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201802_dedup_refi = dedup_result_pred_refi(pred201802_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201802_dedup_refi, 'pred_refiV1_feb18_20190409.csv')

In [ ]:
%%spark

#2018.4
pred201804_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201804_dedup_refi = dedup_result_pred_refi(pred201804_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201804_dedup_refi, 'pred_refiV1_apr18_20190409.csv')

In [ ]:
%%spark

#2018.5
pred201805_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201805_dedup_refi = dedup_result_pred_refi(pred201805_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201805_dedup_refi, 'pred_refiV1_may18_20190409.csv')

In [ ]:
%%spark

#2018.6
pred201806_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201806_dedup_refi = dedup_result_pred_refi(pred201806_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201806_dedup_refi, 'pred_refiV1_jun18_20190409.csv')

In [ ]:
%%spark

#2018.7
pred201807_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201807_dedup_refi = dedup_result_pred_refi(pred201807_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201807_dedup_refi, 'pred_refiV1_jul18_20190409.csv')

In [ ]:
%%spark

#2018.8
pred201808_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201808_dedup_refi = dedup_result_pred_refi(pred201808_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201808_dedup_refi, 'pred_refiV1_aug18_20190409.csv')

In [ ]:
%%spark

#2018.12
pred201812_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201812_dedup_refi = dedup_result_pred_refi(pred201812_refi)

save_results_csv_refi(pred201812_dedup_refi, 'pred_refiV1_dec18_20190205.csv')

#### 2019

In [ ]:
%%spark

#2019.1
pred201901_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201901_dedup_refi = dedup_result_pred_refi(pred201901_refi)

save_results_csv_refi(pred201901_dedup_refi, 'pred_refiV1_jan19_20190215.csv')

In [ ]:
%%spark

#2019.2
pred201902_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201902_dedup_refi = dedup_result_pred_refi(pred201902_refi)

save_results_csv_refi(pred201902_dedup_refi, 'pred_refiV1_feb19_20190305.csv')

In [ ]:
%%spark

#2019.3
pred201903_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201903_dedup_refi = dedup_result_pred_refi(pred201903_refi)

save_results_csv_refi(pred201903_dedup_refi, 'pred_refiV1_mar19_20190409.csv')

In [ ]:
%%spark

#2019.4
pred201904_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201904_dedup_refi = dedup_result_pred_refi(pred201904_refi)

save_results_csv_refi(pred201904_dedup_refi, 'pred_refiV1_apr19_20190507.csv')

In [ ]:
%%spark

#2019.5
pred201905_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201905_dedup_refi = dedup_result_pred_refi(pred201905_refi)

save_results_csv_refi(pred201905_dedup_refi, 'pred_refiV1_2_may19_20190610.csv')

In [ ]:
%%spark

#2019.6
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_jun19_20190717.csv')

In [ ]:
%%spark

#2019.7
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_jul19_20190805.csv')

In [ ]:
%%spark

#2019.8
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_aug19_20190904.csv')

In [ ]:
%%spark

#2019.9
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_sep19_20191004.csv')

In [ ]:
%%spark

#2019.10
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_oct19_20191105.csv')

In [ ]:
%%spark

#2019.11
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_nov19_20191203.csv')

### 2020

In [48]:
%%spark

#2020.01
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_jan20_20200203.csv')

In [57]:
%%spark

#2020.02
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_feb20_20200305.csv')

In [58]:
%%spark

#2020.03
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_mar20_20200406.csv')

In [48]:
%%spark

#2020.04
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_apr20_20200505.csv')

In [57]:
%%spark

#2020.05
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_4_may20_20200610.csv')

In [90]:
%%spark

##2020.05
#save_csv_refi_old(preprocessed_month_data_old, 'pred_refiV1_3_may20_20200610.csv')

In [58]:
%%spark

preprocessed_month_data_refi.show(1)

+-------------------+----------+-------------------+----------+--------+--------+--------+--------------+------------+----------------------+---------------------+--------------------+------------------+----------------------+-------------+-----------------------+--------------------+------------+-------------+------------------+-------------+-------------------+------------------+-----------------+-------------------+---------------+-----------------------+--------------------+---------+-----------------+
|servicecalendardate|     ln_no|         og_note_dt|ct_age_exp|ct_1_exp|ct_2_exp|ct_3_exp|home_value_exp|p_edu_hs_exp|personnum_per_room_exp|mosaic_group_refi_exp|mosaic_group_pur_exp|ratespread_min_exp|ratespread_min_pur_exp|ln_ann_int_rt|loantypedescription_exp|loanamortizationtype|ageinmon_exp|og_mtg_am_exp|   currentcltv_exp|orig_fico_exp|LiveYears_short_exp|LiveYears_long_exp|LiveYears_grp_exp|ln_purpose_type_exp|ln_purpose_type|og_occupy_stat_type_exp|issingleborrower_exp|ln_tr

## 5. Mover Purchase Model Scoring

### (1) Read Models

In [59]:
%%spark

pur_model_pipeline_lr = PipelineModel.load(path = model_save_path_pur + 'MoverV3_3_LR_20200513')
pur_model_pipeline_rf = PipelineModel.load(path = model_save_path_pur + 'MoverV3_3_RF_20200513')
pur_model_pipeline_gbt = PipelineModel.load(path = model_save_path_pur + 'MoverV3_3_GBT_20200513')

### (2) Rename columns to score

In [60]:
%%spark

preprocessed_month_data_pur = preprocessed_month_data

#.withColumn('loantype_exp', F.upper(F.col('loantypedescription_exp')))\
#.withColumnRenamed('ratespread_min_pur_exp', 'ratecompare_exp')\
#.withColumnRenamed('mosaic_group_pur_exp', 'mosaic_group_exp')

### (3) Prediction

In [61]:
%%spark

def fullMonthPrediction_pur(df):
    
    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
    predictionsLR = pur_model_pipeline_lr.transform(df)
    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
                                .select('ln_no', col('pred').alias('logRegProb'))
    
    predictionsRF = pur_model_pipeline_rf.transform(df)
    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
                               .select('ln_no', col('pred').alias('randForProb'))

    predictionsGBT = pur_model_pipeline_gbt.transform(df)
    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
                                .select('ln_no', col('pred').alias('gbtProb'))

    final_join = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
                           .join(pred_df_gbt, on='ln_no', how='left')
     
    final_join = final_join.dropDuplicates()
    
    return final_join

In [62]:
%%spark

def dedup_result_pred_pur(pred_df):
    
    pred_df_dedup = pred_df.groupby('ln_no')\
                            .agg(F.avg('logRegProb').alias('logRegProb'),
                                 F.avg('randForProb').alias('randForProb'),
                                 F.avg('gbtProb').alias('gbtProb')
                            )

    return pred_df_dedup

In [63]:
%%spark

def save_csv_pur(df_prep, filename, result_path = result_path_pur):
    
    pred = fullMonthPrediction_pur(df_prep)
    dedup = dedup_result_pred_pur(pred)
    
    dedup.coalesce(1).write.csv(result_path + filename, header=True)

#### 2018

In [ ]:
%%spark

#2018.1
pred201801_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201801_dedup_pur = dedup_result_pred_pur(pred201801_pur)

save_results_csv_pur(pred201801_dedup_pur, 'pred_Mover_CT3_jan18_20190409.csv')

In [ ]:
%%spark

#2018.2
pred201802_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201802_dedup_pur = dedup_result_pred_pur(pred201802_pur)

save_results_csv_pur(pred201802_dedup_pur, 'pred_Mover_CT3_feb18_20190409.csv')

In [ ]:
%%spark

#2018.4
pred201804_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201804_dedup_pur = dedup_result_pred_pur(pred201804_pur)

save_results_csv_pur(pred201804_dedup_pur, 'pred_Mover_CT3_apr18_20190409.csv')

In [ ]:
%%spark

#2018.5
pred201805_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201805_dedup_pur = dedup_result_pred_pur(pred201805_pur)

save_results_csv_pur(pred201805_dedup_pur, 'pred_Mover_CT3_may18_20190409.csv')

In [ ]:
%%spark

#2018.11
pred201811_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201811_dedup_pur = dedup_result_pred_pur(pred201811_pur)

save_results_csv_pur(pred201811_dedup_pur, 'pred_Mover_CT3_nov18_20190409.csv')

In [ ]:
%%spark

#2018.12
pred201812_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201812_dedup_pur = dedup_result_pred_pur(pred201812_pur)

save_results_csv_pur(pred201812_dedup_pur, 'pred_Mover_CT3_dec18_20190131.csv')

#### 2019

In [ ]:
%%spark

#2019.1
pred201901_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201901_dedup_pur = dedup_result_pred_pur(pred201901_pur)

save_results_csv_pur(pred201901_dedup_pur, 'pred_Mover_CT3_jan19_20190215.csv')

In [ ]:
%%spark

#2019.2
pred201902_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201902_dedup_pur = dedup_result_pred_pur(pred201902_pur)

save_results_csv_pur(pred201902_dedup_pur, 'pred_Mover_CT3_feb19_20190305.csv')

In [ ]:
%%spark

#2019.3
pred201903_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201903_dedup_pur = dedup_result_pred_pur(pred201903_pur)

save_results_csv_pur(pred201903_dedup_pur, 'pred_Mover_CT3_mar19_20190409.csv')

In [ ]:
%%spark

#2019.4
pred201904_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201904_dedup_pur = dedup_result_pred_pur(pred201904_pur)

save_results_csv_pur(pred201904_dedup_pur, 'pred_Mover_CT3_apr19_20190507.csv')

In [ ]:
%%spark

#2019.5
pred201905_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201905_dedup_pur = dedup_result_pred_pur(pred201905_pur)

save_results_csv_pur(pred201905_dedup_pur, 'pred_Mover_CT3_may19_20190610.csv')

In [ ]:
%%spark

#2019.6
pred201906_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201906_dedup_pur = dedup_result_pred_pur(pred201906_pur)

save_results_csv_pur(pred201906_dedup_pur, 'pred_Mover_CT3_june19_20190708.csv')

In [ ]:
%%spark

#2019.7
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_jul19_20190805.csv')

In [ ]:
%%spark

#2019.8
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_aug19_20190904.csv')

In [ ]:
%%spark

#2019.9
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_sep19_20191004.csv')

In [ ]:
%%spark

#2019.10
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_oct19_20191105.csv')

In [ ]:
%%spark

#2019.11
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_nov19_20191203.csv')

### 2020

In [54]:
%%spark

#2020.01
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_jan20_20200203.csv')

In [63]:
%%spark

#2020.02
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_feb20_20200305.csv')

In [65]:
%%spark

#2020.03
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_mar20_20200406.csv')

In [55]:
%%spark

#2020.04
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_apr20_20200505.csv')

In [64]:
%%spark

#2020.05
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_3_may20_20200610.csv')

## 6. Expanded Mover Purchase Models Scoring

### (1). Read Models

In [65]:
%%spark

# 3-6 Months 
model_pipeline_lr_6 = PipelineModel.load(path = model_save_path_pur + 'Mover_3to6_V1_1_LR_20200515')
model_pipeline_rf_6 = PipelineModel.load(path = model_save_path_pur + 'Mover_3to6_V1_1_RF_20200515')
model_pipeline_gbt_6 = PipelineModel.load(path = model_save_path_pur + 'Mover_3to6_V1_1_GBT_20200515')

# 6-12 Months
model_pipeline_lr_12 = PipelineModel.load(path = model_save_path_pur + 'Mover_6to12_V1_1_LR_20200516')
model_pipeline_rf_12 = PipelineModel.load(path = model_save_path_pur + 'Mover_6to12_V1_1_RF_20200516')
model_pipeline_gbt_12 = PipelineModel.load(path = model_save_path_pur + 'Mover_6to12_V1_1_GBT_20200516')

### (2). Prediction

In [66]:
%%spark

preprocessed_month_data.printSchema()

root
 |-- servicecalendardate: date (nullable = true)
 |-- ln_no: string (nullable = true)
 |-- og_note_dt: timestamp (nullable = true)
 |-- ct_age_exp: integer (nullable = true)
 |-- ct_1_exp: string (nullable = true)
 |-- ct_2_exp: string (nullable = true)
 |-- ct_3_exp: string (nullable = true)
 |-- home_value_exp: double (nullable = true)
 |-- p_edu_hs_exp: integer (nullable = true)
 |-- personnum_per_room_exp: double (nullable = true)
 |-- mosaic_group_refi_exp: string (nullable = true)
 |-- mosaic_group_pur_exp: string (nullable = true)
 |-- ratespread_min_exp: double (nullable = true)
 |-- ratespread_min_pur_exp: double (nullable = true)
 |-- ln_ann_int_rt: double (nullable = true)
 |-- loantypedescription_exp: string (nullable = true)
 |-- loanamortizationtype: string (nullable = true)
 |-- ageinmon_exp: double (nullable = true)
 |-- og_mtg_am_exp: double (nullable = true)
 |-- currentcltv_exp: double (nullable = true)
 |-- orig_fico_exp: integer (nullable = true)
 |-- LiveYear

In [67]:
%%spark

def predict(model_pipeline, data):
    
    _scoreUdf = udf(lambda v: float(v[1]), DoubleType())
    
    prediction = model_pipeline.transform(data)
    pred_df = prediction.withColumn('pred', _scoreUdf(prediction['probability']))
    return pred_df

In [68]:
%%spark

def fullMonthPrediction_pur_ex(df, model_pipeline_lr, model_pipeline_rf, model_pipeline_gbt):
    
    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
    predictionsLR = model_pipeline_lr.transform(df)
    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
                                .select('ln_no', 'og_note_dt', col('pred').alias('logRegProb'))
    
    predictionsRF = model_pipeline_rf.transform(df)
    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
                                .select('ln_no', col('pred').alias('randForProb'))
    
    predictionsGBT = model_pipeline_gbt.transform(df)
    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
                                .select('ln_no', col('pred').alias('gbtProb'))
    
    combinedFinalPred = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
                                    .join(pred_df_gbt, on='ln_no', how='left')\
                                    .dropDuplicates()
    
    return combinedFinalPred

In [69]:
%%spark

def dedup_result_pred_pur_ex(pred_df):
    
    pred_df_dedup = pred_df.groupby('ln_no', 'og_note_dt')\
                            .agg(F.avg('logRegProb').alias('logRegProb'),
                                 F.avg('randForProb').alias('randForProb'),
                                 F.avg('gbtProb').alias('gbtProb')
                            )

    return pred_df_dedup

In [70]:
%%spark

def save_csv_pur(df_prep, model_pipeline_lr, model_pipeline_rf, model_pipeline_gbt, filename, result_path):
    
    pred = fullMonthPrediction_pur_ex(df_prep, model_pipeline_lr, model_pipeline_rf, model_pipeline_gbt)
    dedup = dedup_result_pred_pur_ex(pred)
    
    dedup.coalesce(1).write.csv(result_path + filename, header=True)

### (3). 3-6 Months

#### 2020

In [61]:
%%spark

# 2020-01
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_6, model_pipeline_rf_6, model_pipeline_gbt_6,
             'pred_Mover_3to6_V1_jan20_20200203.csv', 
             result_path_pur_6)

In [71]:
%%spark

# 2020-02
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_6, model_pipeline_rf_6, model_pipeline_gbt_6,
             'pred_Mover_3to6_V1_feb20_20200305.csv', 
             result_path_pur_6)

In [71]:
%%spark

# 2020-03
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_6, model_pipeline_rf_6, model_pipeline_gbt_6,
             'pred_Mover_3to6_V1_mar20_20200406.csv', 
             result_path_pur_6)

In [63]:
%%spark

# 2020-04
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_6, model_pipeline_rf_6, model_pipeline_gbt_6,
             'pred_Mover_3to6_V1_apr20_20200505.csv', 
             result_path_pur_6)

In [71]:
%%spark

# 2020-05
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_6, model_pipeline_rf_6, model_pipeline_gbt_6,
             'pred_Mover_3to6_V1_1_may20_20200610.csv', 
             result_path_pur_6)

### (4). 6-12 Months

#### 2020

In [62]:
%%spark

# 2020-01
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_12, model_pipeline_rf_12, model_pipeline_gbt_12,
             'pred_Mover_6to12_V1_jan20_20200203.csv', 
             result_path_pur_12)

In [72]:
%%spark

# 2020-02
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_12, model_pipeline_rf_12, model_pipeline_gbt_12,
             'pred_Mover_6to12_V1_feb20_20200305.csv', 
             result_path_pur_12)

In [72]:
%%spark

# 2020-03
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_12, model_pipeline_rf_12, model_pipeline_gbt_12,
             'pred_Mover_6to12_V1_mar20_20200406.csv', 
             result_path_pur_12)

In [62]:
%%spark

# 2020-04
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_12, model_pipeline_rf_12, model_pipeline_gbt_12,
             'pred_Mover_6to12_V1_apr20_20200505.csv', 
             result_path_pur_12)

In [72]:
%%spark

# 2020-05
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_12, model_pipeline_rf_12, model_pipeline_gbt_12,
             'pred_Mover_6to12_V1_1_may20_20200610.csv', 
             result_path_pur_12)

## 7. Compile Mover Purchase Model Results

### (1). Compile Monthly Results

In [73]:
%%spark

def compile_scores(scorefile_3, scorefile_6, scorefile_12):
    
    df_gcid = spark.sql('select * from data_science_sandbox.gcidservicingloans')
    
    score_3 = spark.read.csv(result_path_pur + scorefile_3, header=True)\
                    .select('ln_no', 
                            col('gbtProb').cast('double').alias('score_0_3mo'))
    score_6 = spark.read.csv(result_path_pur_6 + scorefile_6, header=True)\
                    .select('ln_no', 
                            col('og_note_dt').substr(1, 10).alias('origination_date'),
                            col('gbtProb').cast('double').alias('score_3_6mo'))
    score_12 = spark.read.csv(result_path_pur_12 + scorefile_12, header=True)\
                    .select('ln_no', col('gbtProb').cast('double').alias('score_6_12mo'))
        
    score_all = score_3.join(score_6, on='ln_no', how='inner')\
                        .join(score_12, on='ln_no', how='inner')
        
    ls = []
    for score in ['score_0_3mo', 'score_3_6mo', 'score_6_12mo']:
        ls.extend(statFunc(score_all).approxQuantile(score, [0.7, 0.85, 0.95], 0.00001))
        
    # Top 5% of score_0_3mo are Mover
    # Remaining of top 15% of score_3_6mo are Searcher (~10%)
    # Remaining of top 30% of score_6_12mo are Thinker (~15%)
    expr_type = when(col('score_0_3mo') >= ls[2], 'Mover')\
                    .otherwise(
                        when(col('score_3_6mo') >= ls[4], 'Searcher')\
                            .otherwise(
                                when(col('score_6_12mo') >= ls[6], 'Thinker')\
                                    .otherwise('Other')
                            )
                    )

    score_all = score_all.withColumn('type', expr_type)\
                            .join(df_gcid, score_all.ln_no == df_gcid.Loannumber, how='left')
    
    return score_all

In [74]:
%%spark

def score_save(scorefile_3, scorefile_6, scorefile_12, servicedate, filename):
    
    score_df = compile_scores(scorefile_3, scorefile_6, scorefile_12)
    
    df1 = score_df.withColumn('servicecalendardate', lit(servicedate))\
                    .select('servicecalendardate', 'ln_no', 'GCID', 'origination_date', 
                            'type', 'score_0_3mo', 'score_3_6mo', 'score_6_12mo')
    
    df1.coalesce(1).write.csv(result_path_pur_all + filename, header=True)

In [75]:
%%spark

score_save(
    scorefile_3 = 'pred_MoverV3_3_may20_20200610.csv',
    scorefile_6 = 'pred_Mover_3to6_V1_1_may20_20200610.csv',
    scorefile_12 = 'pred_Mover_6to12_V1_1_may20_20200610.csv',
    servicedate = '2020-05-31',
    filename = 'pred_Mover_Pur_All_may20_20200610.csv'
)

### (2). Insert into the table for Dashboard for Nan

In [76]:
%%spark

# t1 = spark.read.csv(result_path_pur_all + 'pred_Mover_Pur_All_mar20_20200406.csv', header=True).withColumnRenamed('ln_no', 'loan')

# print(t1.count())
# t1.show(1)

In [81]:
%%spark

# t1.groupby('type').count().show()

In [78]:
%%spark

# First snapshot: 10/31/2019
# Going forward, insert into the table, instead of overwriting.

# t1.write.mode('append').saveAsTable('Analytical.wol_dashboard_serviced_purchase_v1')

In [79]:
%%spark

# df = spark.sql('''
# select *
# from Analytical.wol_dashboard_serviced_purchase_v1
# ''')

# df.show()

## 8. Save Results into Prod Path

### (1). Save Model Results

In [92]:
%%spark

## download pred_refiV1_3_oct19_20191105.csv and pred_MoverV3_2_oct19_20191105.csv, upload on DW, let Lu do the rest
# result_path_refi = '/dev/projects/retention_models/refi_payoff/training/results/'
# result_path_pur = '/dev/projects/retention_models/purchase_payoff/training/results/movermodel/'

score_refi = spark.read.csv(result_path_refi + 'pred_refiV1_4_may20_20200610.csv', header=True)\
                    .select(col('ln_no').alias('loan'), col('gbtProb').alias('score'))
        
score_pur = spark.read.csv(result_path_pur + 'pred_MoverV3_3_may20_20200610.csv', header=True)\
                    .select(col('ln_no').alias('loan'), col('gbtProb').alias('score'))

print(score_refi.count())
print(score_pur.count())

score_refi.show(1)
score_pur.show(1)

1889579
1889579
+----------+------------------+
|      loan|             score|
+----------+------------------+
|3433101386|0.6133600376423959|
+----------+------------------+
only showing top 1 row

+----------+-----------------+
|      loan|            score|
+----------+-----------------+
|3429917877|0.262411623881287|
+----------+-----------------+
only showing top 1 row

In [94]:
%%spark

## use snapshotdate in the file name
score_refi.coalesce(1).write.csv(prod_refi_path + 'leadgen_servicing_mover_refi_20200610.csv', header=True)
score_pur.coalesce(1).write.csv(prod_pur_path + 'leadgen_servicing_mover_pur_20200610.csv', header=True)

### (2). Save score path, and update with new version model 

In [95]:
%%spark

refi_file = spark.createDataFrame([('/prod/projects/model_conformed/RocketScience/LeadGen/RefiLeadGen/1.0/leadgen_servicing_mover_refi_20200610.csv',)], ['filepath'])
pur_file = spark.createDataFrame([('/prod/projects/model_conformed/RocketScience/LeadGen/PurLeadGen/1.0/leadgen_servicing_mover_pur_20200610.csv',)], ['filepath'])

refi_file.show(10, False)
pur_file.show(10, False)

+--------------------------------------------------------------------------------------------------------------+
|filepath                                                                                                      |
+--------------------------------------------------------------------------------------------------------------+
|/prod/projects/model_conformed/RocketScience/LeadGen/RefiLeadGen/1.0/leadgen_servicing_mover_refi_20200610.csv|
+--------------------------------------------------------------------------------------------------------------+

+------------------------------------------------------------------------------------------------------------+
|filepath                                                                                                    |
+------------------------------------------------------------------------------------------------------------+
|/prod/projects/model_conformed/RocketScience/LeadGen/PurLeadGen/1.0/leadgen_servicing_mover_pur_2020

In [96]:
%%spark

# Before running, delete the two csv files in Ambari.
file_path = '/prod/projects/model_conformed/RocketScience/LeadGen/Currentfile/'

refi_file.coalesce(1).write.csv(file_path + 'current_leadgen_servicing_mover_refi.csv', header=True)
pur_file.coalesce(1).write.csv(file_path + 'current_leadgen_servicing_mover_pur.csv', header=True)

## save these two files and let BV do the rest

# Actual

In [ ]:
%%spark

from pyspark.sql.functions import lower

csv_path = '/dev/projects/retention_models/csv_data/'
actual_path = '/dev/projects/retention_models/actual_value/'
monthly_path = '/dev/projects/retention_models/monthly_snapshot/'

In [ ]:
%%spark
 
df_actual = spark.read.csv(csv_path + 'Payoff_segments_summary_20191206.csv', header=True)\
                        .withColumn('newloanpurpose', lower(col('NewLoanPurpose')))\
                        .withColumn('retentiontype', lower(col('PayoffRetentionType')))\
                        .select('ServicedLoanNumber', 'PaymentInFullDate', 'newloanpurpose', 'retentiontype')
    
df_list1 = spark.sql('''
select distinct servicecalendardate, ln_no
from data_science_sandbox.servicing_dbo_payoff_source_purchase_client_crdtrs 
where leftportfoliodate is null
and servicecalendardate between '2016-06-30' and '2018-11-30'
''')

## upload from DW table

df_201812 = spark.read.parquet(monthly_path+'servicing_df_all_dec18_20190131.parquet').select('servicecalendardate', 'ln_no')
 
df_201901 = spark.read.parquet(monthly_path+'servicing_df_all_jan19_20190215.parquet').select('servicecalendardate', 'ln_no')
df_201902 = spark.read.parquet(monthly_path+'servicing_df_all_feb19_20190305.parquet').select('servicecalendardate', 'ln_no')
df_201903 = spark.read.parquet(monthly_path+'servicing_df_all_mar19_20190409.parquet').select('servicecalendardate', 'ln_no')
df_201904 = spark.read.parquet(monthly_path+'servicing_df_all_apr19_20190507.parquet').select('servicecalendardate', 'ln_no')
df_201905 = spark.read.parquet(monthly_path+'servicing_df_all_may19_20190610.parquet').select('servicecalendardate', 'ln_no')
df_201906 = spark.read.parquet(monthly_path+'servicing_df_all_june19_20190708.parquet').select('servicecalendardate', 'ln_no')
df_201907 = spark.read.parquet(monthly_path+'servicing_df_all_jul19_20190805.parquet').select('servicecalendardate', 'ln_no')
df_201908 = spark.read.parquet(monthly_path+'servicing_df_all_aug19_20190904.parquet').select('servicecalendardate', 'ln_no')
df_201909 = spark.read.parquet(monthly_path+'servicing_df_all_sep19_20191004.parquet').select('servicecalendardate', 'ln_no')
df_201910 = spark.read.parquet(monthly_path+'servicing_df_all_oct19_20191105.parquet').select('servicecalendardate', 'ln_no')
df_201911 = spark.read.parquet(monthly_path+'servicing_df_all_nov19_20191202.parquet').select('servicecalendardate', 'ln_no')

 
df_list = df_list1.union(df_201812)\
                    .union(df_201901).union(df_201902).union(df_201903)\
                    .union(df_201904).union(df_201905).union(df_201906)\
                    .union(df_201907).union(df_201908).union(df_201909)\
                    .union(df_201910).union(df_201911)
 

In [ ]:
%%spark

df_payoff = df_list.join(df_actual, df_actual.ServicedLoanNumber == df_list.ln_no, how='left')
 
def build_targets(df):
    
    # Purchase Payoff
    expr_purpayoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('newloanpurpose')=='purchase'), 1.)\
                        .otherwise(0.)
    expr_purpayoff_3_6 = when((col('PaymentInFullDate')>col('enddt_3')) & (col('PaymentInFullDate')<=col('enddt_6')) & (col('newloanpurpose')=='purchase'), 1.)\
                            .otherwise(0.)
    expr_purpayoff_6_12 = when((col('PaymentInFullDate')>col('enddt_6')) & (col('PaymentInFullDate')<=col('enddt_12')) & (col('newloanpurpose')=='purchase'), 1.)\
                            .otherwise(0.)
    expr_purpayoff_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('newloanpurpose')=='purchase'), 1.)\
                            .otherwise(0.)
    
    # Refi Payoff
    expr_refipayoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('newloanpurpose')!='purchase'), 1.)\
                        .otherwise(0.)
    expr_refipayoff_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('newloanpurpose')!='purchase'), 1.)\
                            .otherwise(0.)
    
    # Purchase/Refi Retained
    expr_purretain_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) 
                             & (col('newloanpurpose')=='purchase') & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    
    expr_refiretain_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) 
                             & (col('newloanpurpose')!='purchase') & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    
    # Payoff
    expr_payoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')), 1.)\
                        .otherwise(0.)
    expr_payoff_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')), 1.)\
                        .otherwise(0.)
    expr_payoff_all = when(col('PaymentInFullDate')>col('servicecalendardate'), 1.)\
                        .otherwise(0.)
    
    # Retained
    expr_retention = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('retentiontype')=='retained'), 1.)\
                        .otherwise(0.)
    expr_retention_3_6 = when((col('PaymentInFullDate')>col('enddt_3')) & (col('PaymentInFullDate')<=col('enddt_6')) & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    expr_retention_6_12 = when((col('PaymentInFullDate')>col('enddt_6')) & (col('PaymentInFullDate')<=col('enddt_12')) & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    expr_retention_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) & (col('retentiontype')=='retained'), 1.)\
                        .otherwise(0.)
    expr_retention_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('retentiontype')=='retained'), 1.)\
                        .otherwise(0.)
 
    df_final = df.withColumn('startdt', F.add_months(col('servicecalendardate'), 1))\
                    .withColumn('enddt_3', F.add_months(col('servicecalendardate'), 4))\
                    .withColumn('enddt_6', F.add_months(col('servicecalendardate'), 7))\
                    .withColumn('enddt_12', F.add_months(col('servicecalendardate'), 13))\
                    .withColumn('enddt_24', F.add_months(col('servicecalendardate'), 25))\
                    .withColumn('purchasepayoff', expr_purpayoff)\
                    .withColumn('purchasepayoff_3_6', expr_purpayoff_3_6)\
                    .withColumn('purchasepayoff_6_12', expr_purpayoff_6_12)\
                    .withColumn('purchasepayoff_all', expr_purpayoff_all)\
                    .withColumn('refipayoff', expr_refipayoff)\
                    .withColumn('refipayoff_all', expr_refipayoff_all)\
                    .withColumn('purretain_24', expr_purretain_24)\
                    .withColumn('refiretain_24', expr_refiretain_24)\
                    .withColumn('payoff', expr_payoff)\
                    .withColumn('payoff_24', expr_payoff_24)\
                    .withColumn('payoff_all', expr_payoff_all)\
                    .withColumn('retained', expr_retention)\
                    .withColumn('retained_3_6', expr_retention_3_6)\
                    .withColumn('retained_6_12', expr_retention_6_12)\
                    .withColumn('retained_24', expr_retention_24)\
                    .withColumn('retained_all', expr_retention_all)\
                    .select('servicecalendardate', 'ln_no', 'PaymentInFullDate', 'newloanpurpose', 'retentiontype',
                            'purchasepayoff', 'purchasepayoff_3_6', 'purchasepayoff_6_12', 'purchasepayoff_all',
                            'refipayoff', 'refipayoff_all', 
                            'purretain_24', 'refiretain_24',
                            'payoff', 'payoff_24', 'payoff_all', 
                            'retained', 'retained_3_6', 'retained_6_12', 'retained_24', 'retained_all')
                        
    return df_final
 

In [ ]:
%%spark

df_final = build_targets(df_payoff)
 
df_final.write.parquet(actual_path + 'Actual_payoff_20191206.parquet', mode='overwrite')


## Check the count

In [ ]:
%%spark

df1 = spark.read.parquet(actual_path + 'Actual_payoff_20191106.parquet')
df1.groupBy('servicecalendardate').count().sort(desc("servicecalendardate")).show(3)

In [ ]:
%%spark

df2 = spark.read.parquet(actual_path + 'Actual_payoff_20191206.parquet')
df2.groupBy('servicecalendardate').count().sort(desc("servicecalendardate")).show(4)

In [ ]:
%%spark

df1.show(1)
df2.show(1)